# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nhl_correlation(): 
    
    ##CREATE FUNCTION TO RETURN CLEANED DF
    def nhl():
        nhl_df=pd.read_csv("assets/nhl.csv")
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        #remove spaces from the headers of both data frames 
        nhl_df=nhl_df.rename(mapper=str.strip,axis='columns')
        cities=cities.rename(mapper=str.strip,axis='columns')
        #filter out win-loss ratio for 2018
        nhl_df=nhl_df[nhl_df['year']==2018]
        #clean up nhl team names from wiki info
        cities['NHL'].replace("\[[\w\s]*\]",value="",inplace=True,regex=True)
        #drop columns with a dash or that are empty
        cities=cities[cities['NHL']!='—'][cities['NHL']!='']
        #extract out required columns from the cities dataframe 
        cities=cities.iloc[:,[0,1,5]]
        #rename columns in cities dataframe 
        cities.rename(columns={'Population (2016 est.)[8]':'Population','NHL':'team'},inplace=True)
        #filter out rows with divison name inside 
        #apply function on all columns of df
        nhl_df=nhl_df[nhl_df['team'].str.contains("Division")==False]
        #get win-loss ratio 
        nhl_df.iloc[:,1:]=nhl_df.iloc[:,1:-1].apply(pd.to_numeric,axis=0)
        nhl_df["Ratio"]=nhl_df["W"]/(nhl_df['L']+nhl_df['W'])
        #Extract out required columns from nhl_df
        nhl_df=nhl_df[['team','Ratio','W','L']]
        #Remove asterisks from team name in nhl_df
        nhl_df['team'].replace('\*',value='',inplace=True,regex=True)
        #extract out the last word in the nhl_df dataset for teams
        nhl_df['team']=nhl_df['team'].str.extract('([\w]+$)')
        #extract out the last word in the cities dataset for teams so that it can match
        #with nhl_df
        cities['team']=cities['team'].str.extract('([A-Z]{1}[\w]+$)')


        ##Prepare to join data
        #set indexes of both dataframes to team
        nhl_df=nhl_df.set_index('team')
        cities=cities.set_index('team')
        #join both nhl_df and cities dataset 
        sub=pd.merge(cities,nhl_df,how='left',right_index=True,left_index=True)

        #Input values for missing values by calculating team averages 
        nyc=nhl_df.loc[['Rangers','Islanders','Devils']]
        la=nhl_df.loc[['Kings','Ducks']]
        #calculate averages by converting win and loss column to numeric
        nyc_val=(nyc['W']/(nyc['W']+nyc['L'])).mean()
        la_val=(la['W']/(la['W']+la['L'])).mean()
        #input values
        sub.loc[['RangersIslandersDevils','KingsDucks'],['Ratio']] = [nyc_val,la_val]
        sub['Population']=pd.to_numeric(sub['Population'])
        sub=sub.loc[:,['Metropolitan area','Ratio','Population']]
        sub.set_index('Metropolitan area',inplace=True)
        return sub
    
    ##CALCULATE CORR
    sub=nhl()
    #perform calculations based on sub dataframe
    population_by_region = sub['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = sub['Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


0.012486162921209912

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nba_correlation():
    
    ##CREATE FUNCTION TO RETURN CLEANED DF
    def nba():
        nba_df=pd.read_csv("assets/nba.csv")
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]

        #remove spaces from the headers of both data frames 
        nba_df=nba_df.rename(mapper=str.strip,axis='columns')
        cities=cities.rename(mapper=str.strip,axis='columns')
        #filter out win-loss ratio for 2018
        nba_df=nba_df[nba_df['year']==2018]
        #extract out required columns 
        nba_df=nba_df.iloc[:,0:3]
        #convert win and loss to numeric 
        nba_df[['W','L']]=nba_df[['W','L']].apply(pd.to_numeric)
        #clean names by removing asterisks and brackers from the end 
        nba_df['team'].replace('([\*]?\s\([\w]+\))',regex=True,inplace=True,value='')
        #extract out last word from the team name 
        nba_df['team']=nba_df['team'].str.extract('([\w]+$)')
        #calculate win-loss ratio
        nba_df['Ratio']=nba_df['W']/(nba_df['W']+nba_df['L'])

        #Start cleaning cities dataframe
        #rename population
        cities.rename(columns={'Population (2016 est.)[8]':'Population','NBA':'team'},inplace=True)
        #extract out required columns 
        cities=cities.iloc[:,[0,1,4]]
        #clean up the team names by removing [] 
        cities['team'].replace('(\[[\w\s]*\])',value='',inplace=True,regex=True)
        #drop columns with -- or that are empty 
        cities=cities[(cities['team']!='')&(cities['team']!='—')]
        #convert populations to int 
        cities['Population']=pd.to_numeric(cities['Population'])

        ##JOIN DATAFRAMES BASED ON MATCHING ENTRIES 
        #set indexes to be the same (team)
        nba_df.set_index('team',inplace=True)
        cities.set_index('team',inplace=True)
        sub=pd.merge(cities,nba_df,how='left',left_index=True,right_index=True)
        #calculate nyc and la values, together with blazers
        nyc=nba_df.loc[['Knicks','Nets']]
        la=nba_df.loc[['Lakers','Clippers']]
        blazers=nba_df.loc['Blazers']
        #calculate nyc and la values 
        nyc_val=nyc['Ratio'].mean()
        la_val=la['Ratio'].mean()
        #input nyc and la values 
        sub.loc[['KnicksNets','LakersClippers','Trail Blazers'],'Ratio']=[nyc_val,la_val,blazers['Ratio']]
        sub=sub[['Metropolitan area','Ratio','Population']]
        sub.set_index('Metropolitan area',inplace=True)
        return sub

    ##CALCULATE CORR
    sub=nba()
    population_by_region = sub['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = sub['Ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


-0.1765716025284462

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def mlb_correlation(): 
    
    ##CREATE FUNCTION TO RETURN CLEANED DF
    def mlb():
        mlb_df=pd.read_csv("assets/mlb.csv")
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]

        #remove spacing from column headers of both dataframes
        mlb_df=mlb_df.rename(mapper=str.strip,axis='columns')
        cities=cities.rename(mapper=str.strip,axis='columns')
        #select year
        mlb_df=mlb_df[mlb_df['year']==2018]

        #start cleaning mlb_df
        #extract out required columns 
        mlb_df=mlb_df.iloc[:,0:3]
        #Calculate win loss ratio
        mlb_df['Ratio']=mlb_df['W']/(mlb_df['W']+mlb_df['L'])
        #extract out last word to match 
        mlb_df['n_team']=mlb_df['team'].str.extract('([\w]+(?=\s))')
        #rename n_team to appropriate values 
        mlb_df['n_team']=mlb_df['n_team'].replace({'New':'New York City','Tampa':'Tampa Bay Area',
                                                  'Los':'Los Angeles','Louis':'St. Louis',
                                                  'Kansas':'Kansas City','Miami':'Miami–Fort Lauderdale',
                                                  'Washington':'Washington, D.C.','Arizona':'Phoenix',
                                                  'Minnesota':'Minneapolis–Saint Paul','Texas':'Dallas–Fort Worth',
                                                   'Colorado':'Denver','Oakland':'San Francisco Bay Area'})
        #Replace visibly missing values for teams
        mlb_df.iloc[-2:,4]=['San Francisco Bay Area','San Diego']

        #start cleaning cities df 
        #extract out required columns 
        cities=cities.iloc[:,[0,1,3]]
        #rename population column and MLB column
        cities.rename(columns={'Population (2016 est.)[8]':'Population','MLB':'team'},inplace=True)
        #convet population column to numeric 
        cities['Population']=pd.to_numeric(cities['Population'])
        #clean team column by removing []
        cities['team'].replace('(\[[\w\s]*\])',regex=True,inplace=True,value='')
        #drop empty rows and rows with -- for team column values
        cities=cities[(cities['team']!='')&(cities['team']!='—')]

        #join both dataframes
        #set dataframe indexes to be the same(team)
        mlb_df.set_index('n_team',inplace=True)
        cities.set_index('Metropolitan area',inplace=True)
        sub=pd.merge(cities,mlb_df,how='left',left_index=True,right_index=True)
        #groupby area and calculate mean ratio
        sub.reset_index(inplace=True)
        sub.rename(columns={'index':'Metropolitan area'},inplace=True)
        final=sub.groupby('Metropolitan area').agg(Ratio=('Ratio',np.mean))
        final=pd.merge(final,cities,how='left',left_index=True,right_index=True)
        final=final.iloc[:,:2]
        return final
    
    ##CALCULATE CORR
    final=mlb()
    population_by_region = final['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = final['Ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 
    
    ##CREATE FUNCTION TO RETURN CLEANED DF
    def nfl():
        nfl_df=pd.read_csv("assets/nfl.csv")
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]

        ##clean nfl_df
        #remove spacing from column name of both dataframes
        nfl_df=nfl_df.rename(mapper=str.strip,axis='columns') 
        cities=cities.rename(mapper=str.strip,axis='columns')
        #select required columns from nfl_df
        nfl_df=nfl_df[['team','W','L','year']]
        #remove string rows
        nfl_df=nfl_df[nfl_df['W'].str.contains("NFC")==False]
        nfl_df=nfl_df[nfl_df['W'].str.contains("AFC")==False]
        #convert W and L columns to numeric 
        nfl_df[['W','L']]=nfl_df[['W','L']].apply(pd.to_numeric)
        #find win-loss ratio
        nfl_df['Ratio']=nfl_df['W']/(nfl_df['W']+nfl_df['L'])
        #clean up team names by removing * and +
        nfl_df['team'].replace('[\*\+]',value='',regex=True,inplace=True)
        #select the correct year from nfl_df (2018)
        nfl_df=nfl_df[nfl_df['year']==2018]
        #extract last word from teams to match with cities team names 
        nfl_df['code']=nfl_df['team'].str.extract('([\w]+$)')

        ##clean cities df
        #extract required columns 
        cities=cities.iloc[:,0:3]
        cities.rename(columns={'Population (2016 est.)[8]':'Population','NFL':'team'},inplace=True)
        #convert population to numeric 
        cities['Population']=pd.to_numeric(cities['Population'])
        #clean team names by removing brackets
        cities['team'].replace('\[[\w\s]*\]',value='',inplace=True,regex=True)
        #remove blank spaces and line entries 
        cities=cities[(cities['team']!='')&(cities['team']!='—')]
        #drop toronto
        cities.drop(13,inplace=True)

        ##join both dataframes 
        #set index to team
        nfl_df.set_index('code',inplace=True)
        cities.set_index('team',inplace=True)
        sub=pd.merge(cities,nfl_df,how='left',left_index=True,right_index=True)
        #find values for nyc,la and san francisco
        req=['Giants','Jets','Rams','Chargers','49ers','Raiders']
        to_calc=nfl_df.loc[req]
        to_calc['loc']=['NYC','NYC','LA','LA','SF','SF']
        #do groupby and find mean
        grouped_vals=to_calc.groupby('loc').agg(avg=('Ratio',np.mean))
        #substitute values back into sub dataframe
        la_val,nyc_val,sf_val=grouped_vals.iloc[0][0],grouped_vals.iloc[1][0],grouped_vals.iloc[2][0]
        sub.loc[['GiantsJets','RamsChargers','49ersRaiders'],'Ratio']=nyc_val,la_val,sf_val
        sub=sub[['Metropolitan area','Ratio','Population','team']]
        sub.set_index('Metropolitan area',inplace=True)
        sub=sub.iloc[:,:2]
        return sub 

    ##CALCULATE CORR
    sub=nfl()
    population_by_region = sub['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = sub['Ratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


0.004922112149349386

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def sports_team_performance():
    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    ##LOAD ALL REQUIRED FUNCTIONS THAT RETURN CLEANED DATASETS (PREVENTS AUTOGRADER ERROR MESSAGE)
    ##COPY FUNCTIONS OVER FROM PREVO
    
    ##NFL
    def nfl():
        nfl_df=pd.read_csv("assets/nfl.csv")
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]

        ##clean nfl_df
        #remove spacing from column name of both dataframes
        nfl_df=nfl_df.rename(mapper=str.strip,axis='columns') 
        cities=cities.rename(mapper=str.strip,axis='columns')
        #select required columns from nfl_df
        nfl_df=nfl_df[['team','W','L','year']]
        #remove string rows
        nfl_df=nfl_df[nfl_df['W'].str.contains("NFC")==False]
        nfl_df=nfl_df[nfl_df['W'].str.contains("AFC")==False]
        #convert W and L columns to numeric 
        nfl_df[['W','L']]=nfl_df[['W','L']].apply(pd.to_numeric)
        #find win-loss ratio
        nfl_df['Ratio']=nfl_df['W']/(nfl_df['W']+nfl_df['L'])
        #clean up team names by removing * and +
        nfl_df['team'].replace('[\*\+]',value='',regex=True,inplace=True)
        #select the correct year from nfl_df (2018)
        nfl_df=nfl_df[nfl_df['year']==2018]
        #extract last word from teams to match with cities team names 
        nfl_df['code']=nfl_df['team'].str.extract('([\w]+$)')

        ##clean cities df
        #extract required columns 
        cities=cities.iloc[:,0:3]
        cities.rename(columns={'Population (2016 est.)[8]':'Population','NFL':'team'},inplace=True)
        #convert population to numeric 
        cities['Population']=pd.to_numeric(cities['Population'])
        #clean team names by removing brackets
        cities['team'].replace('\[[\w\s]*\]',value='',inplace=True,regex=True)
        #remove blank spaces and line entries 
        cities=cities[(cities['team']!='')&(cities['team']!='—')]
        #drop toronto
        cities.drop(13,inplace=True)

        ##join both dataframes 
        #set index to team
        nfl_df.set_index('code',inplace=True)
        cities.set_index('team',inplace=True)
        sub=pd.merge(cities,nfl_df,how='left',left_index=True,right_index=True)
        #find values for nyc,la and san francisco
        req=['Giants','Jets','Rams','Chargers','49ers','Raiders']
        to_calc=nfl_df.loc[req]
        to_calc['loc']=['NYC','NYC','LA','LA','SF','SF']
        #do groupby and find mean
        grouped_vals=to_calc.groupby('loc').agg(avg=('Ratio',np.mean))
        #substitute values back into sub dataframe
        la_val,nyc_val,sf_val=grouped_vals.iloc[0][0],grouped_vals.iloc[1][0],grouped_vals.iloc[2][0]
        sub.loc[['GiantsJets','RamsChargers','49ersRaiders'],'Ratio']=nyc_val,la_val,sf_val
        sub=sub[['Metropolitan area','Ratio','Population','team']]
        sub.set_index('Metropolitan area',inplace=True)
        sub=sub.iloc[:,:2]
        return sub 
    
    ##NBA
    def nba():
        nba_df=pd.read_csv("assets/nba.csv")
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]

        #remove spaces from the headers of both data frames 
        nba_df=nba_df.rename(mapper=str.strip,axis='columns')
        cities=cities.rename(mapper=str.strip,axis='columns')
        #filter out win-loss ratio for 2018
        nba_df=nba_df[nba_df['year']==2018]
        #extract out required columns 
        nba_df=nba_df.iloc[:,0:3]
        #convert win and loss to numeric 
        nba_df[['W','L']]=nba_df[['W','L']].apply(pd.to_numeric)
        #clean names by removing asterisks and brackers from the end 
        nba_df['team'].replace('([\*]?\s\([\w]+\))',regex=True,inplace=True,value='')
        #extract out last word from the team name 
        nba_df['team']=nba_df['team'].str.extract('([\w]+$)')
        #calculate win-loss ratio
        nba_df['Ratio']=nba_df['W']/(nba_df['W']+nba_df['L'])

        #Start cleaning cities dataframe
        #rename population
        cities.rename(columns={'Population (2016 est.)[8]':'Population','NBA':'team'},inplace=True)
        #extract out required columns 
        cities=cities.iloc[:,[0,1,4]]
        #clean up the team names by removing [] 
        cities['team'].replace('(\[[\w\s]*\])',value='',inplace=True,regex=True)
        #drop columns with -- or that are empty 
        cities=cities[(cities['team']!='')&(cities['team']!='—')]
        #convert populations to int 
        cities['Population']=pd.to_numeric(cities['Population'])

        ##JOIN DATAFRAMES BASED ON MATCHING ENTRIES 
        #set indexes to be the same (team)
        nba_df.set_index('team',inplace=True)
        cities.set_index('team',inplace=True)
        sub=pd.merge(cities,nba_df,how='left',left_index=True,right_index=True)
        #calculate nyc and la values, together with blazers
        nyc=nba_df.loc[['Knicks','Nets']]
        la=nba_df.loc[['Lakers','Clippers']]
        blazers=nba_df.loc['Blazers']
        #calculate nyc and la values 
        nyc_val=nyc['Ratio'].mean()
        la_val=la['Ratio'].mean()
        #input nyc and la values 
        sub.loc[['KnicksNets','LakersClippers','Trail Blazers'],'Ratio']=[nyc_val,la_val,blazers['Ratio']]
        sub=sub[['Metropolitan area','Ratio','Population']]
        sub.set_index('Metropolitan area',inplace=True)
        return sub
    
    ##NHL
    def nhl():
        nhl_df=pd.read_csv("assets/nhl.csv")
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        #remove spaces from the headers of both data frames 
        nhl_df=nhl_df.rename(mapper=str.strip,axis='columns')
        cities=cities.rename(mapper=str.strip,axis='columns')
        #filter out win-loss ratio for 2018
        nhl_df=nhl_df[nhl_df['year']==2018]
        #clean up nhl team names from wiki info
        cities['NHL'].replace("\[[\w\s]*\]",value="",inplace=True,regex=True)
        #drop columns with a dash or that are empty
        cities=cities[cities['NHL']!='—'][cities['NHL']!='']
        #extract out required columns from the cities dataframe 
        cities=cities.iloc[:,[0,1,5]]
        #rename columns in cities dataframe 
        cities.rename(columns={'Population (2016 est.)[8]':'Population','NHL':'team'},inplace=True)
        #filter out rows with divison name inside 
        #apply function on all columns of df
        nhl_df=nhl_df[nhl_df['team'].str.contains("Division")==False]
        #get win-loss ratio 
        nhl_df.iloc[:,1:]=nhl_df.iloc[:,1:-1].apply(pd.to_numeric,axis=0)
        nhl_df["Ratio"]=nhl_df["W"]/(nhl_df['L']+nhl_df['W'])
        #Extract out required columns from nhl_df
        nhl_df=nhl_df[['team','Ratio','W','L']]
        #Remove asterisks from team name in nhl_df
        nhl_df['team'].replace('\*',value='',inplace=True,regex=True)
        #extract out the last word in the nhl_df dataset for teams
        nhl_df['team']=nhl_df['team'].str.extract('([\w]+$)')
        #extract out the last word in the cities dataset for teams so that it can match
        #with nhl_df
        cities['team']=cities['team'].str.extract('([A-Z]{1}[\w]+$)')


        ##Prepare to join data
        #set indexes of both dataframes to team
        nhl_df=nhl_df.set_index('team')
        cities=cities.set_index('team')
        #join both nhl_df and cities dataset 
        sub=pd.merge(cities,nhl_df,how='left',right_index=True,left_index=True)

        #Input values for missing values by calculating team averages 
        nyc=nhl_df.loc[['Rangers','Islanders','Devils']]
        la=nhl_df.loc[['Kings','Ducks']]
        #calculate averages by converting win and loss column to numeric
        nyc_val=(nyc['W']/(nyc['W']+nyc['L'])).mean()
        la_val=(la['W']/(la['W']+la['L'])).mean()
        #input values
        sub.loc[['RangersIslandersDevils','KingsDucks'],['Ratio']] = [nyc_val,la_val]
        sub['Population']=pd.to_numeric(sub['Population'])
        sub=sub.loc[:,['Metropolitan area','Ratio','Population']]
        sub.set_index('Metropolitan area',inplace=True)
        return sub
    
    ##MLB
    def mlb():
        mlb_df=pd.read_csv("assets/mlb.csv")
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]

        #remove spacing from column headers of both dataframes
        mlb_df=mlb_df.rename(mapper=str.strip,axis='columns')
        cities=cities.rename(mapper=str.strip,axis='columns')
        #select year
        mlb_df=mlb_df[mlb_df['year']==2018]

        #start cleaning mlb_df
        #extract out required columns 
        mlb_df=mlb_df.iloc[:,0:3]
        #Calculate win loss ratio
        mlb_df['Ratio']=mlb_df['W']/(mlb_df['W']+mlb_df['L'])
        #extract out last word to match 
        mlb_df['n_team']=mlb_df['team'].str.extract('([\w]+(?=\s))')
        #rename n_team to appropriate values 
        mlb_df['n_team']=mlb_df['n_team'].replace({'New':'New York City','Tampa':'Tampa Bay Area',
                                                  'Los':'Los Angeles','Louis':'St. Louis',
                                                  'Kansas':'Kansas City','Miami':'Miami–Fort Lauderdale',
                                                  'Washington':'Washington, D.C.','Arizona':'Phoenix',
                                                  'Minnesota':'Minneapolis–Saint Paul','Texas':'Dallas–Fort Worth',
                                                   'Colorado':'Denver','Oakland':'San Francisco Bay Area'})
        #Replace visibly missing values for teams
        mlb_df.iloc[-2:,4]=['San Francisco Bay Area','San Diego']

        #start cleaning cities df 
        #extract out required columns 
        cities=cities.iloc[:,[0,1,3]]
        #rename population column and MLB column
        cities.rename(columns={'Population (2016 est.)[8]':'Population','MLB':'team'},inplace=True)
        #convet population column to numeric 
        cities['Population']=pd.to_numeric(cities['Population'])
        #clean team column by removing []
        cities['team'].replace('(\[[\w\s]*\])',regex=True,inplace=True,value='')
        #drop empty rows and rows with -- for team column values
        cities=cities[(cities['team']!='')&(cities['team']!='—')]

        #join both dataframes
        #set dataframe indexes to be the same(team)
        mlb_df.set_index('n_team',inplace=True)
        cities.set_index('Metropolitan area',inplace=True)
        sub=pd.merge(cities,mlb_df,how='left',left_index=True,right_index=True)
        #groupby area and calculate mean ratio
        sub.reset_index(inplace=True)
        sub.rename(columns={'index':'Metropolitan area'},inplace=True)
        final=sub.groupby('Metropolitan area').agg(Ratio=('Ratio',np.mean))
        final=pd.merge(final,cities,how='left',left_index=True,right_index=True)
        final=final.iloc[:,:2]
        return final
    
    
    ##CODE FOR QN 5 STARTS HERE
    
    #assign variables to the correct dataframe 
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    baseball,hockey,basketball,football=mlb(),nhl(),nba(),nfl()
    #rename dataframe ratio column to reflect league name
    baseball.rename(columns={'Ratio':'Ratio_b'},inplace=True)
    hockey.rename(columns={'Ratio':'Ratio_h'},inplace=True)
    football.rename(columns={'Ratio':'Ratio_f'},inplace=True)
    basketball.rename(columns={'Ratio':'Ratio_bb'},inplace=True)
    to_iterate=[football,basketball,hockey,baseball]
    #create dataframe to replace the values
    vals=pd.DataFrame(columns=sports,index=sports)
    for i in range(len(to_iterate)):
        for g in range(len(to_iterate)):
            #do inner join of dataframes 
            calc=pd.merge(to_iterate[i].iloc[:,0],to_iterate[g].iloc[:,0],how='inner',left_index=True,right_index=True)
            #find p value by passing dataframe into function
            sub=stats.ttest_rel(calc.iloc[:,0],calc.iloc[:,1])
            vals.iloc[i,g],vals.iloc[g,i]=sub[1],sub[1]
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    #df.loc["NFL","NFL"] should return np.nan
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    p_values=vals.apply(pd.to_numeric)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
    

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
